<a href="https://colab.research.google.com/github/Cynthiaudoye/Forex_Trading_Bot/blob/main/trading_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from tensorflow.keras.models import load_model
from joblib import load
import datetime

# Load the models
best_model_gru = load_model('/content/drive/MyDrive/Ms.c Data Science project/best_model_gru.keras')
best_model_xgb = xgb.Booster()
best_model_xgb.load_model('/content/drive/MyDrive/Ms.c Data Science project/best_model_xgb.json')
best_model_rf = load('/content/drive/MyDrive/Ms.c Data Science project/best_model_rf.pkl')

# Load the scaler
scaler_X = load('/content/drive/MyDrive/Ms.c Data Science project/scaler_X.bin')
scaler_y = load('/content/drive/MyDrive/Ms.c Data Science project/scaler_y.bin')

# Function to fetch real-time Forex data using Yahoo Finance
def fetch_real_time_data_yahoo(symbol, period='1d', interval='1m'):
    data = yf.download(tickers=symbol, period=period, interval=interval)
    if data.empty:
        return None
    data.reset_index(inplace=True)
    return data

# Fetch the latest data for the past 15 minutes
symbol = 'GBPUSD=X'  # Yahoo Finance symbol for GBP/USD
latest_market_data = fetch_real_time_data_yahoo(symbol, period='1d', interval='1m')

# Check if data was fetched successfully
if latest_market_data is None:
    print("Failed to fetch data. Ensure the symbol is correct and try again.")
else:
    # Calculate necessary technical indicators
    latest_market_data.loc[:, 'EMA_20'] = latest_market_data['Close'].ewm(span=20).mean()
    latest_market_data.loc[:, 'EMA_50'] = latest_market_data['Close'].ewm(span=50).mean()

    # RSI Calculation
    delta = latest_market_data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    latest_market_data.loc[:, 'RSI'] = 100 - (100 / (1 + rs))
    latest_market_data['RSI'].fillna(0, inplace=True)  # Fill NaN values with zeros

    # MACD Calculation
    short_ema = latest_market_data['Close'].ewm(span=12, adjust=False).mean()
    long_ema = latest_market_data['Close'].ewm(span=26, adjust=False).mean()
    latest_market_data.loc[:, 'MACD_Line'] = short_ema - long_ema
    latest_market_data.loc[:, 'Signal_Line'] = latest_market_data['MACD_Line'].ewm(span=9, adjust=False).mean()
    latest_market_data.loc[:, 'MACD_Histogram'] = latest_market_data['MACD_Line'] - latest_market_data['Signal_Line']

    # Verify the data and check for NaN values
    print("Fetched Data:\n", latest_market_data.tail())
    print("NaN Values:\n", latest_market_data.isna().sum())

    # Create sequences for the last 15 time steps
    def create_sequences(data, time_steps=15):
        Xs = []
        for i in range(len(data) - time_steps + 1):
            v = data.iloc[i:(i + time_steps)].values
            Xs.append(v)
        return np.array(Xs)

    features = ['Open', 'High', 'Low', 'EMA_20', 'EMA_50', 'RSI', 'MACD_Line', 'Signal_Line', 'MACD_Histogram']
    latest_sequences = create_sequences(latest_market_data[features], time_steps=15)
    print("Sequences Shape:", latest_sequences.shape)

    # Scale the latest data sequences
    latest_sequences_scaled = np.array([scaler_X.transform(seq) for seq in latest_sequences])

    # Make predictions using the ensemble model
    gru_prediction = best_model_gru.predict(latest_sequences_scaled)
    gru_prediction = scaler_y.inverse_transform(gru_prediction).flatten()

    # Flatten the sequences for XGBoost and Random Forest
    latest_sequences_flat = latest_sequences_scaled.reshape(latest_sequences_scaled.shape[0], -1)
    xgb_prediction = best_model_xgb.predict(xgb.DMatrix(latest_sequences_flat))
    xgb_prediction = scaler_y.inverse_transform(xgb_prediction.reshape(-1, 1)).flatten()

    rf_prediction = best_model_rf.predict(latest_sequences_flat)
    rf_prediction = scaler_y.inverse_transform(rf_prediction.reshape(-1, 1)).flatten()

    ensemble_prediction = np.mean([gru_prediction, xgb_prediction, rf_prediction], axis=0)

    # Inspect final predictions
    print("GRU Prediction:", gru_prediction[-1])
    print("XGB Prediction:", xgb_prediction[-1])
    print("RF Prediction:", rf_prediction[-1])
    print("Ensemble Prediction:", ensemble_prediction[-1])

    # Generate trading signal based on the prediction
    def generate_signal(prediction, current_price):
        if prediction > current_price:
            return 'Buy'
        else:
            return 'Sell'

    current_price = latest_market_data['Close'].values[-1]
    signal = generate_signal(ensemble_prediction[-1], current_price)

    # Create a DataFrame for the signal
    today = datetime.datetime.now()
    today_str = today.strftime('%Y.%m.%d')

    signal_df = pd.DataFrame({
        'Date': [today_str],
        'Signal': [signal]
    })

    # Save the signal to CSV
    signal_df.to_csv('trading_signals.csv', index=False)

    # Display the signal DataFrame
    print(signal_df)


[*********************100%%**********************]  1 of 1 completed


Fetched Data:
                      Datetime      Open      High       Low     Close  \
635 2024-07-19 10:51:00+01:00  1.291189  1.291189  1.291189  1.291189   
636 2024-07-19 10:52:00+01:00  1.291072  1.291072  1.291072  1.291072   
637 2024-07-19 10:53:00+01:00  1.290756  1.290756  1.290756  1.290756   
638 2024-07-19 10:54:00+01:00  1.290756  1.290756  1.290756  1.290756   
639 2024-07-19 10:55:00+01:00  1.290722  1.290722  1.290722  1.290722   

     Adj Close  Volume    EMA_20    EMA_50        RSI  MACD_Line  Signal_Line  \
635   1.291189       0  1.291251  1.291429  52.832479  -0.000036    -0.000062   
636   1.291072       0  1.291234  1.291415  53.849326  -0.000046    -0.000059   
637   1.290756       0  1.291188  1.291389  37.689119  -0.000079    -0.000063   
638   1.290756       0  1.291147  1.291364  39.404117  -0.000104    -0.000071   
639   1.290722       0  1.291107  1.291339  28.820749  -0.000126    -0.000082   

     MACD_Histogram  
635        0.000026  
636        0.00